# Set references for v8 tuning and export to ONNX


In [1]:
from kolmov import crossval_table, crossval_fit_table, get_color_fader, fit_table
from Gaugi import mkdir_p
from sklearn.model_selection import StratifiedKFold, KFold
from multiprocessing import Process
from time import sleep
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import json
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, model_from_json


from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/fitting'
mkdir_p(output_path)

In [3]:
def create_op_dict(op):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              'fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              'sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              'pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              'fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              'sp_val'    : "reference/"+op+"_cutbased/sp_val",
              'pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              'fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              'sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              'pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              'fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              'pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              'fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              'pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              'fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              'pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              'fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              'pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              'fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              'pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              'fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op)

In [4]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
min_avgmu = 16
max_avgmu = 100

## 1) Reading all tunings:


In [5]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [6]:
cv.from_csv('output/crossval/table_v8.csv')

In [7]:
best_inits = cv.filter_inits("max_sp_val")
best_inits = best_inits.loc[(best_inits.model_idx==3)] # 5 neurons for all phase spaces
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

## 2) Linear correction:

In [8]:
basepath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.npz'
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

In [9]:
def my_data_generator( path, pidname='el_lhmedium'):  
    import pandas as pd
    from kepler import load
    df = load(path)
    # NOTE: Offline filter lhvloose -> lhmedium (as the training procedure)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | (df.target==0) ]
    return df


def my_input_generator( df ):
    
    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)
    
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    avgmu = df['avgmu'].values.astype(np.float32)
    target = df['target'].values.astype(np.int16)
    rings = norm1(rings)
    return [rings]

def my_dec_generator_after( row, df , remove_last=True):
    
    from Gaugi import load as gload
    file_name = row.file_name.values[0]
    tuned_idx = row.tuned_idx.values[0]
    model_idx = row.model_idx.values[0]
    slope = row.slope.values[0]
    offset = row.offset.values[0]
    tuned = gload(file_name)['tunedData'][tuned_idx]
    model = model_from_json( json.dumps(tuned['sequence'], separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( tuned['weights'] )
    if remove_last:
        model = Model(model.inputs, model.layers[-2].output)
        
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    avgmu = df.avgmu.values.flatten()
    avgmu[avgmu>max_avgmu] = max_avgmu
    avgmu[avgmu<min_avgmu] = min_avgmu
    dec = np.greater(output, slope*avgmu + offset)
    return dec
    
def my_dec_generator( row, df ):
    
    from Gaugi import load as gload
    file_name = row.file_name.values[0]
    tuned_idx = row.tuned_idx.values[0]
    model_idx = row.model_idx.values[0]
    tuned = gload(file_name)['tunedData'][tuned_idx]
    model = model_from_json( json.dumps(tuned['sequence'], separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( tuned['weights'] )
    thr = tuned['history']['summary']['threshold_op']
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    dec = np.greater(output, thr)
    return dec

In [10]:
# calculate all pd/fa from reference file
ref_path = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins/'
ref_path += 'references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_values = [[ {} for eta in range(5)] for et in range(5)]

from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name + '_cutbased')/refObj.getSgnTotal(op_name + '_cutbased')
            _fa = refObj.getBkgPassed(op_name + '_cutbased')/refObj.getBkgTotal(op_name + '_cutbased')
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa, 'pd_epsilon':0.0}

In [11]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

**NOTE**: This step will take something like 20min to be ready

In [12]:
cv_fit = crossval_fit_table( etbins, etabins, kf, my_data_generator, my_input_generator )

In [13]:
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
1214,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.978118,...,232819,187639,23000,657,23282,18764,229996,6862,232819,187639
1213,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.978118,...,232819,187639,22778,445,23282,18764,227780,4979,232819,187639
1212,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.978118,...,232819,187639,22760,435,23282,18764,227618,4873,232819,187639
1215,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.978118,...,232819,187639,23015,683,23282,18764,230153,7089,232819,187639
1274,v8,0,0,3,1,7,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.975846,...,232819,187639,23000,715,23282,18764,229997,6831,232819,187639


In [14]:
best_inits_after = cv_fit.fill( best_inits , paths, ref_values, output_path=output_path ,
                                min_avgmu=min_avgmu, max_avgmu=max_avgmu)

Running (et=4, eta4): 100%|██████████| 10/10 [01:14<00:00,  7.49s/it]


In [15]:
cv_fit.to_csv('output/fitting/best_inits_v8_after.csv')
best_inits_after = pd.read_csv('output/fitting/best_inits_v8_after.csv')

In [16]:
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
1214,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.978118,...,232819,187639,23000,657,23282,18764,229996,6862,232819,187639
1213,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.978118,...,232819,187639,22778,445,23282,18764,227780,4979,232819,187639
1212,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.978118,...,232819,187639,22760,435,23282,18764,227618,4873,232819,187639
1215,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.978118,...,232819,187639,23015,683,23282,18764,230153,7089,232819,187639
1274,v8,0,0,3,1,7,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.975846,...,232819,187639,23000,715,23282,18764,229997,6831,232819,187639


In [17]:
best_inits_after.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background
0,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.978118,...,23282,18764,229821,6479,232818,187639,-0.011726,0.358803,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...
1,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.978118,...,23282,18764,227486,4663,232818,187639,-0.012003,0.745003,output/fitting/figures/correction_signal_mediu...,output/fitting/figures/correction_background_m...
2,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.978118,...,23282,18764,227263,4546,232818,187639,-0.012125,0.772200,output/fitting/figures/correction_signal_tight...,output/fitting/figures/correction_background_t...
3,v8,0,0,3,0,4,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.978118,...,23282,18764,229989,6683,232818,187639,-0.011855,0.326329,output/fitting/figures/correction_signal_vloos...,output/fitting/figures/correction_background_v...
4,v8,0,0,3,1,7,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.975846,...,23282,18764,229867,6517,232818,187639,-0.011331,0.337474,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...


### 4) Run test:

In [18]:
basepath = '/home/jodafons/public/cern_data/new_files/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.npz'
test_paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

### 4.1) Before correction:

In [19]:
test_table = cv.evaluate( best_sorts , test_paths, my_data_generator, my_dec_generator)

Fitting... : 100%|████████████████████| 25/25 [07:26<00:00, 17.87s/it]


In [20]:
for op_name in op_names:
    table = best_inits.loc[best_inits.op_name == op_name]
    output = 'output/fitting/table_v8_%s.pdf'%op_name
    cv.dump_beamer_table(table, output, tags=['v8'], 
                         test_table = test_table.loc[test_table.op_name==op_name],
                         title='data17_13TeV %s v8 tuning (with tanh)'%op_name)

2021-12-16 17:00:28,040 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_tight.pdf latex code...
2021-12-16 17:00:30,007 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_medium.pdf latex code...
2021-12-16 17:00:31,822 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_loose.pdf latex code...
2021-12-16 17:00:33,649 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_vloose.pdf latex code...


### 4.2) After correction:

In [21]:
best_inits_after['delta_ref'] = abs(best_inits_after.pd_ref - best_inits_after.pd_op)
best_sorts_after = cv.filter_sorts(best_inits_after, 'delta_ref', idxmin=True )
best_sorts_after.to_csv('output/fitting/best_sorts_after.csv', index=False)

In [22]:
best_sorts_after.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background,delta_ref
28,v8,0,0,3,7,1,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.976713,...,18763,229935,6485,232818,187639,-0.011552,0.335296,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...,0.000259
9,v8,0,0,3,2,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.975065,...,18764,227671,4678,232818,187639,-0.012346,0.781103,output/fitting/figures/correction_signal_mediu...,output/fitting/figures/correction_background_m...,0.000467
10,v8,0,0,3,2,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.975065,...,18764,227510,4605,232818,187639,-0.012579,0.810642,output/fitting/figures/correction_signal_tight...,output/fitting/figures/correction_background_t...,0.000465
31,v8,0,0,3,7,1,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.976713,...,18763,230080,6661,232818,187639,-0.011798,0.305857,output/fitting/figures/correction_signal_vloos...,output/fitting/figures/correction_background_v...,0.000308
64,v8,0,1,3,6,9,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.966565,...,14366,138996,7559,140971,143657,-0.012458,0.237809,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...,0.000681


In [23]:
test_table_after = cv.evaluate( best_sorts_after , test_paths, my_data_generator, my_dec_generator_after )

Fitting... : 100%|████████████████████| 25/25 [07:20<00:00, 17.64s/it]


In [24]:
test_table.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,fa_op_passed,pd_op_total,fa_op_total,pd_test,fa_test,sp_test,pd_test_passed,pd_test_total,fa_test_passed,fa_test_total
0,v8,0,0,3,5,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.977071,...,6766,232819,187639,0.985064,0.035244,0.974883,234330,237883,5052,143342
1,v8,0,0,3,5,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.977071,...,4903,232819,187639,0.985064,0.035244,0.974883,234330,237883,5052,143342
2,v8,0,0,3,5,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.977071,...,4805,232819,187639,0.985064,0.035244,0.974883,234330,237883,5052,143342
3,v8,0,0,3,5,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.977071,...,6952,232819,187639,0.985064,0.035244,0.974883,234330,237883,5052,143342
4,v8,0,1,3,5,0,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.966040,...,7970,141000,143657,0.981588,0.049103,0.966181,141919,144581,5414,110257


In [25]:
test_table.to_csv('output/fitting/table_v8_test_after.csv', index=False)

In [26]:
for op_name in op_names:
    table = best_inits_after.loc[best_inits_after.op_name == op_name]
    output = 'output/fitting/table_v8_%s_after.pdf'%op_name
    cv.dump_beamer_table(table, output, tags=['v8'], 
                         test_table = test_table.loc[test_table.op_name==op_name],
                         title='data17_13TeV %s v8 tuning (with correction)'%op_name)

2021-12-16 17:07:56,850 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_tight_after.pdf latex code...
2021-12-16 17:07:58,720 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_medium_after.pdf latex code...
2021-12-16 17:08:00,571 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_loose_after.pdf latex code...
2021-12-16 17:08:02,411 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file output/fitting/table_v8_vloose_after.pdf latex code...


## 5) Export all tunings:

In [27]:
best_sorts_after.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background,delta_ref
28,v8,0,0,3,7,1,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.976713,...,18763,229935,6485,232818,187639,-0.011552,0.335296,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...,0.000259
9,v8,0,0,3,2,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,medium,0.975065,...,18764,227671,4678,232818,187639,-0.012346,0.781103,output/fitting/figures/correction_signal_mediu...,output/fitting/figures/correction_background_m...,0.000467
10,v8,0,0,3,2,2,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,tight,0.975065,...,18764,227510,4605,232818,187639,-0.012579,0.810642,output/fitting/figures/correction_signal_tight...,output/fitting/figures/correction_background_t...,0.000465
31,v8,0,0,3,7,1,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,vloose,0.976713,...,18763,230080,6661,232818,187639,-0.011798,0.305857,output/fitting/figures/correction_signal_vloos...,output/fitting/figures/correction_background_v...,0.000308
64,v8,0,1,3,6,9,/home/jodafons/public/tuning_data/Zee/v8/r0/us...,0,loose,0.966565,...,14366,138996,7559,140971,143657,-0.012458,0.237809,output/fitting/figures/correction_signal_loose...,output/fitting/figures/correction_background_l...,0.000681


In [28]:
model_name_format = 'data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v8.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
op_capnames = ['Tight', 'Medium', 'Loose', 'VeryLoose']
for idx, op in enumerate( ['tight','medium','loose','vloose'] ):
    
    cv_fit.export(best_sorts_after.loc[best_sorts_after.op_name==op], 
                  model_name_format.format(op=op_capnames[idx]), 
                  config_name_format.format(op=op_capnames[idx]), 
                  op, 
                  to_onnx     = True,
                  remove_last = True,
                  min_avgmu   = min_avgmu,
                  max_avgmu   = max_avgmu)

2021-12-16 17:09:02,374 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerTightTriggerConfig.conf.
2021-12-16 17:10:01,116 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerMediumTriggerConfig.conf.
2021-12-16 17:10:59,413 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerLooseTriggerConfig.conf.
2021-12-16 17:11:57,468 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerVeryLooseTriggerConfig.conf.


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The ma

In [29]:
!mv models output/fitting
!mv *.conf output/fitting
!mv *.tex output/fitting
!mv *.pdf output/fitting

mv: cannot stat '*.tex': No such file or directory
